In [1]:
import numpy as np
from sklearn.datasets import load_iris

In [2]:
# Construct a list that contains the number of neurons of each layer in your network.
structure = input("Enter the number of neurons of each layer in your network").split(' ')
structure = [int(x) for x in structure]
print(structure)    

[4, 2, 3]


In [3]:
# Construct a list that contains all the bias vectors of your network(_n means it has all vectors)
B_n = [np.random.randn(l, 1) for l in structure[1:]]
print("bias vector of each layer\n")
for B in B_n:
    print(B)
    print()

# Construct a list that contains all the weight matrices of your network(_n means it has all matrices)
W_n = [np.random.randn(l, next_l) for l, next_l in zip(structure[:-1], structure[1:])]
print("weight matrix of each layer\n")
for W in W_n:
    print(W)
    print()

bias vector of each layer

[[-0.6379268 ]
 [-0.59526757]]

[[ 1.38966199]
 [-0.11246295]
 [-1.30218694]]

weight matrix of each layer

[[ 0.08290516  1.15085031]
 [-0.46114854 -0.34309745]
 [-0.01330717 -1.15759124]
 [-1.68601068 -0.13406605]]

[[-0.01714642  0.74301235 -0.81068458]
 [ 0.85924933 -0.72574961  0.17620808]]



##### Load training and test datasets

In [4]:
load_iris().keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [5]:
data = load_iris()['data']
target = load_iris()['target']

print(type(data), type(target))
print(data.shape, target.shape)
print(len(data), len(target))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(150, 4) (150,)
150 150


In [6]:
indexes = [0, 52, 108, 49, 142, 88]

data = data[indexes]
target = target[indexes]

print(data)
print(target)

[[5.1 3.5 1.4 0.2]
 [6.9 3.1 4.9 1.5]
 [6.7 2.5 5.8 1.8]
 [5.  3.3 1.4 0.2]
 [5.8 2.7 5.1 1.9]
 [5.6 3.  4.1 1.3]]
[0 1 2 0 2 1]


In [7]:
# Randomly shuffle the data
np.random.seed(123)
np.random.shuffle(data)

np.random.seed(123)
np.random.shuffle(target)

print(data)
print(target)   #2

[[6.9 3.1 4.9 1.5]
 [5.  3.3 1.4 0.2]
 [5.8 2.7 5.1 1.9]
 [5.1 3.5 1.4 0.2]
 [6.7 2.5 5.8 1.8]
 [5.6 3.  4.1 1.3]]
[1 0 2 0 2 1]


##### Forward Pass

In [8]:
# Provide sigmoid and sigmoid_derivative function
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1.0 - sigmoid(x))

In [20]:
X = data[0]
print(X)

y = np.array([0, 1, 0])
print(y)

[6.9 3.1 4.9 1.5]
[0 1 0]


In [10]:
np.array(B_n[0]).reshape(-1)

array([-0.6379268 , -0.59526757])

In [11]:
np.dot(W_n[0].T, X) + np.array(B_n[0]).reshape(-1)

array([-4.08966281,  0.40870132])

In [33]:
# Create two lists that contain pre and post activation vector of each layer, respectively
Z_n, A_n = [], []

# Forward pass layer by layer from L=0(First hidden layer) thru L=H(Output layer)
for i, (b, W) in enumerate(zip(B_n, W_n)):
    if i == 0:
        z = np.dot(np.array(W).T, X) + np.array(b).reshape(-1)
    else:
        z = np.dot(np.array(W).T, a) + np.array(b).reshape(-1)
    a = sigmoid(z)

    Z_n.append(z)
    A_n.append(a)

In [34]:
for z, a in zip(Z_n, A_n):
    print(np.round(z, 4))
    print(np.round(a, 4))
    print()

[-4.0897  0.4087]
[0.0165 0.6008]

[ 1.9056 -0.5362 -1.2097]
[0.8705 0.3691 0.2298]



In [35]:
H = len(structure) - 2
deltas = []

for L in range(H, -1, -1):
    if L == H:
        deltas[L] = sigmoid_derivative(Z_n[L]) * (A_n[L] - y)        
    else:
        deltas[L] = sigmoid_derivative(Z_n[L]) * np.dot(W_n[L+1], deltas[L+1])
        
deltas  

IndexError: list assignment index out of range

In [31]:
H

1

In [26]:
print(deltas)

[array([ 0.09811868, -0.14691716,  0.04066019]), array([-0.00232936,  0.04751267])]


In [24]:
deltas[::-1]

[array([-0.00232936,  0.04751267]),
 array([ 0.09811868, -0.14691716,  0.04066019])]

In [ ]:
e_Je_W, e_Je_B = [], []

for L in range(H, -1, -1):
    if L == H:
        e_Je_W       

In [ ]:
for L in range(H, -1, -1):
            if L != H:
                delta = sigmoid_derivative(Z_n[L]) * np.dot(self.W_n[L+1], delta)
            else:
                delta = sigmoid_derivative(Z_n[L]) * (A_n[L] - y)
                
if L != 0:
                e_Je_W_ns[L] = np.dot(A_n[L-1], delta.T)    
            else:
                e_Je_W_ns[L] = np.dot(x, delta.T) 

In [54]:
B_n

[array([[-0.07351368],
        [ 1.81403277],
        [-0.44199761]]),
 array([[ 1.38951142],
        [-1.07746533],
        [ 0.23848917]])]

In [14]:
q = np.array([[ 0.08290516,  1.15085031], [-0.46114854, -0.34309745],
              [-0.01330717, -1.15759124], [-1.68601068, -0.13406605]])

r = np.array([-0.6379268, -0.59526757])

t = np.dot(q.T, X) + r
u = sigmoid(t)

print(t)
print(u)

[-4.08966282  0.40870132]
[0.01646911 0.60077644]
